In [58]:
import json
import pandas as pd

source_root = "/home/shubham/fabric_main/chat-with-your-data/chat_backend/sample_database/prequel_ai_4521/"
dest_root = '/home/shubham/fabric_main/chat-with-your-data/chat_backend/sample_database/'

In [59]:
# file_name = '/home/shubham/fabric_main/VXcis/VXcis Col desc Tables.JSON'
# with open(file_name, 'r') as f:
#     data = json.load(f)

In [60]:
# len(data)

In [61]:
# data[0][0]['UACTPRM'][0]['ACCOUNT_AGREEMENT_ID']

In [62]:
df = pd.read_csv(source_root+'column_description.csv')
df.head()

,TABLE_NAME,COLUMN_NAME,DESCRIPTION,Values_sample
0,UACTPRM,ACCOUNT_AGREEMENT_ID,Account Agreement Id (DECIMAL): A unique ident...,"11461.00, 26377.00"
1,UACTPRM,ACCOUNT_PREMISES_ID,Account Agreement / Premises Id (BIGINT): A un...,"1, 2"
2,UACTPRM,END_DATE,End Date when account-premises agreement went ...,"1976-04-06, 1976-06-03"
3,UACTPRM,PREMISES_ID,Premises Id (CHAR): A unique identifier for ea...,"1002, 1004"
4,UACTPRM,START_DATE,Start Date (DATE): The date when the account-p...,"1999-09-02, 2000-02-23"


Column Description

In [63]:
col_desc = {}

for i in range(len(df)):

    if df['TABLE_NAME'][i] not in col_desc.keys():
        col_desc[df['TABLE_NAME'][i]]={}

        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]] = {}
        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]]['description'] = df['DESCRIPTION'][i]
        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]]['values_sample'] = [df['Values_sample'][i]]

    else:
        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]] = {}
        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]]['description'] = df['DESCRIPTION'][i]
        col_desc[df['TABLE_NAME'][i]][df['COLUMN_NAME'][i]]['values_sample'] = [df['Values_sample'][i]]


In [64]:
col_desc

{'UACTPRM': {'ACCOUNT_AGREEMENT_ID': {'description': 'Account Agreement Id (DECIMAL): A unique identifier for the account-premises agreement, represented as a decimal number to ensure precise identification.',
   'values_sample': ['11461.00, 26377.00']},
  'ACCOUNT_PREMISES_ID': {'description': 'Account Agreement / Premises Id (BIGINT): A unique identifier combining account and premises details, used to link the account with the specific premises. The BIGINT data type allows for large integer values.',
   'values_sample': ['1, 2']},
  'END_DATE': {'description': 'End Date when account-premises agreement went inactive (NULL when active) (DATE): The date when the account-premises agreement was terminated. NULL indicates that the agreement is still active. The DATE data type captures this date in a standard format.',
   'values_sample': ['1976-04-06, 1976-06-03']},
  'PREMISES_ID': {'description': 'Premises Id (CHAR): A unique identifier for each premises, stored as a fixed-length charact

Table Description

In [65]:
fii=source_root+'tables_description.csv'

tdf = pd.read_csv(fii)
tdf.head()

,Table Name,Table Description,Long Table Descriptions
0,UACTPRM,VXcis: Account / Premises Agreement Table,This is the Account / Premises Agreement Table...
1,UBLH,VXcis: Bill History Table - Summary,"This table is the Bill History Summary table, ..."
2,UCLS,VXcis: Bill Class Table,This is the Bill Class Table. This is a config...
3,UPRM,VXcis: Premises Master Table,This is the Master Premises Table. This table ...
4,UPST,VXcis: Premises Status Type Table,This is the Premises Status Type Table. This i...


In [66]:
table_desc = {}
for i in range(len(tdf)):
    table_desc[tdf['Table Name'][i]] = tdf['Long Table Descriptions'][i]

table_desc

{'UACTPRM': 'This is the Account / Premises Agreement Table. This table is used to identify the association of the account and premises. Using the start and end date you can identify when each account moved in and moved out of a given premises. ',
 'UBLH': 'This table is the Bill History Summary table, it is used to store all the master data for the bill print history.  This detail includes account data such as name, Account ID, Customer ID, DBA, Bill Class, and Mailing address. It also includes Premises details such as service address. Along with a summary of charges and account balances.',
 'UCLS': 'This is the Bill Class Table. This is a configuration table to define the possible options for Bill Class and set any attributes related to that bill class type. ',
 'UPRM': 'This is the Master Premises Table. This table is used to store all the premises is in the database.  A Premises is a location where a utility company delivers a utility service such as electricity, natural gas, water

In [67]:
SAMPLE_QUERIES = [
    {
        "question": "List of services with unbilled usage older than 2 months.",
        "query": "select SERVICE_ID as ServiceId, USAGE_PERIOD as UsagePeriod, BILLING_USAGE as BillingUsage, STATUS_TYPE as Status from UUSEHST where BILL_ID is null and STATUS_TYPE = 'READY' and to_date(cast(usage_period as char(6))||'01','YYYYMMDD') < DATE(SUBSTR(CHAR(now()), 1, 7) || '-01') - 2 months",
        "logic": "This query identifies services with unbilled usage that's older than two months. It checks for records in the 'UUSEHST' table where the 'BILL_ID' is null (indicating unbilled usage), the 'STATUS_TYPE' is 'READY', and the usage period is older than two months from the beginning of the current month.",
        "response": "Here's the list of services with unbilled usage older than 2 months.",
        "chart": {
            "type": "table",
            "x_axis": ["ServiceId", "UsagePeriod", "BillingUsage", "Status"],
            "y_axis": [],
            "logic": "A table is appropriate for displaying a list of services with their corresponding details.",
        },
    },
    {
        "question": "Give me the total number of active electric services.",
        "query": "SELECT a.SERVICE_TYPE AS ServiceType, b.DESCRIPTION AS ServiceTypeDescription, COUNT(DISTINCT a.SERVICE_ID) AS TotalServices FROM uservice a JOIN usvctyp b ON a.SERVICE_TYPE = b.SERVICE_TYPE WHERE b.DESCRIPTION LIKE '%Electric%' AND a.PHYSICAL_DISCONNECT_DATE IS NULL GROUP BY a.SERVICE_TYPE, b.DESCRIPTION",
        "logic": "This query counts the total number of active electric services. It joins the 'uservice' and 'usvctyp' tables to filter for services with a description containing 'Electric' and having a null 'PHYSICAL_DISCONNECT_DATE', indicating active services. The query groups the results by service type and description, providing a count of distinct service IDs.",
        "response": "The total number of active electric services is displayed below.",
        "chart": {
            "type": "bar",
            "x_axis": ["SERVICETYPE"],
            "y_axis": ["TOTALSERVICES"],
            "logic": "Type of services against number of services better represented by bar chart",
        },
    },
    {
        "question": "Give me a list of premises where the route stop doesnt match the service stop.",
        "query": "select a.SERVICE_ID as ServiceId, a.PREMISES_ID as PremisesId, a.METER_READING_STOP_NUMBER as ServiceRoute, UPSTP as PremisesRoute from uservice a join uprm b on a.PREMISES_ID = b.UPPRM where a.METER_READING_STOP_NUMBER <> b.UPSTP",
        "logic": "This query joins the 'uservice' and 'uprm' tables on the 'PREMISES_ID' to compare the 'METER_READING_STOP_NUMBER' (service stop) with the 'UPSTP' (premises route stop). It filters for records where these two values are not equal, indicating a mismatch between the service stop and the premises route stop.",
        "response": "Here is a list of premises where the route stop doesn't match the service stop.",
        "chart": {
            "type": "table",
            "x_axis": ["ServiceId", "PremisesId", "ServiceRoute", "PremisesRoute"],
            "y_axis": [],
            "logic": "A table is suitable for displaying a list of premises with their corresponding service and premises route stop information, allowing for easy comparison and identification of mismatches.",
        },
    },
    {
        "question": "Last bill date of Account 881 / Premises '10000514'?",
        "query": "SELECT a.bhsact AS AccountId, a.bhsprm AS PremisesId, MAX(a.bhsbld) AS LastBillDate FROM ublh a JOIN uprm b ON a.bhsprm = b.upprm WHERE a.bhscrc NOT IN ('C', 'S') AND a.bhsact = 881 AND a.bhsprm like '%10000514%' GROUP BY a.bhsact, a.bhsprm",
        "logic": "This query retrieves the last billing date for a specific account ('881') and premises ('10000514'). It joins the 'ublh' and 'uprm' tables to match the account and premises IDs. The billing status filter excludes canceled ('C') or suppressed ('S') bills.",
        "response": "The last bill date for Account '881'/ Premises '10000514' is displayed below.",
        "chart": {
            "type": "null",
            "x_axis": ["null"],
            "y_axis": ["null"],
            "logic": "Since the question asks for a single date, no chart is needed.",
        },
    },
    {
        "question": "Compare 2023 and 2022 residential water billed usage",
        "query": "SELECT SUBSTR(bhsrym, 1, 4) AS YEAR, SUM(billing_usage) AS BILLINGUSAGE FROM uusehst u JOIN uservice s ON u.service_id = s.service_id JOIN usvctyp t ON s.service_type = t.service_type JOIN ublh b ON u.bill_id = b.bhstr# JOIN ucls c ON b.bhscls = c.uccls WHERE status_type = 'BILLED' AND bhscrc NOT IN ('C', 'S') AND ucdes LIKE '%Residential%' AND t.utility_type = 'W'   AND SUBSTR(bhsrym, 1, 4) IN ('2022', '2023') GROUP BY SUBSTR(bhsrym, 1, 4) ORDER BY SUBSTR(bhsrym, 1, 4)",
        "logic": "This query compares the residential water billed usage between the years 2023 and 2022. It joins multiple tables ('uusehst', 'uservice', 'usvctyp', 'ublh', and 'ucls') to filter the data based on the service type, billing status, utility type, and description. The query sums up the billing usage for the years 2023 and 2022 using conditional aggregation. The 'GROUP BY' clause ensures that the results are grouped by the years in question.",
        "response": "The comparison of residential water billed usage for the years 2023 and 2022 is displayed below.",
        "chart": {
            "type": "bar",
            "x_axis": ["YEAR"],
            "y_axis": ["BILLINGUSAGE"],
            "logic": "A bar chart is appropriate to visualize the comparison of water usage between two years, showing the total billed usage for each year.",
        },
    },
    {
        "question": "What was the billed revenue for water residential accounts for Revenue Period June 2023?",
        "query": "select b.bhsrym as RevenuePeriod, sum(b.bhstot) as BillingRevenue from uusehst u join uservice s on u.service_id = s.service_id join usvctyp t on s.service_type = t.service_type join ublh b on u.bill_id = b.bhstr# join ucls c on b.bhscls = c.uccls where status_type = 'BILLED' and bhscrc not in ('C', 'S') and ucdes like '%Residential%' and bhsrym = 202306 and t.utility_type = 'W' group by b.bhsrym",
        "logic": "This query calculates the total billed revenue for water services for residential accounts within a specific revenue period ('202306'). It joins relevant tables to filter for billed water usage, residential customer class, and the specified revenue period. The 'SUM' function calculates the total revenue from the 'bhstot' column in the 'ublh' table.",
        "response": "The billed revenue for water residential accounts for Revenue Period 'June 2023' is displayed below.",
        "chart": {
            "type": "null",
            "x_axis": ["null"],
            "y_axis": ["null"],
            "logic": "Since the question asks for a single total value, no chart is needed.",
        },
    },
    {
        "question": "What is total number of active premises with active services",
        "query": "SELECT COUNT(DISTINCT UPRM.UPPRM) AS TotalActivePremisesWithActiveServices FROM UPRM INNER JOIN USERVICE ON UPRM.UPPRM = USERVICE.PREMISES_ID INNER JOIN UPST ON UPRM.UPSTS = UPST.UIPSC WHERE UPST.UIIAP <> 'Y'   AND USERVICE.PHYSICAL_DISCONNECT_DATE IS NULL",
        "logic": "This query counts the total number of active premises by joining the UPRM and UPST tables on the premise status ID. It filters out premises where the 'Inactive Premises' flag (UIIAP) in the UPST table is not set to 'Y', indicating active premises.",
        "response": "The total number of active premises is displayed below.",
        "chart": {
            "type": "null",
            "x_axis": ["null"],
            "y_axis": ["null"],
            "logic": "Since the question asks for a single count value, no chart is needed.",
        },
    },
    {
        "question": "What is the total count of discontinued dates populated?",
        "query": "select count(*) as TotalDiscontinued from UPRM where UPCDT <> 0",
        "logic": "This query counts the total number of records in the UPRM table where the discontinued date (UPCDT) is not equal to 0. This indicates the number of records that have a populated discontinued date.",
        "response": "The total count of discontinued dates populated is displayed below.",
        "chart": {
            "type": "null",
            "x_axis": ["null"],
            "y_axis": ["null"],
            "logic": "Since the question asks for a single count value, no chart is needed.",
        },
    },
]


{
    "question": "How is residential billed water usage trending for 2023",
    "query": "select u.usage_period as UsagePeriod, sum(u.billing_usage) as BillingUsage from uusehst u join uservice s on u.service_id = s.service_id join usvctyp t on s.service_type = t.service_type join ublh b on u.bill_id = b.bhstr# join ucls c on b.bhscls = c.uccls where status_type = 'BILLED' and bhscrc not in ('C', 'S') and ucdes like '%Residential%' and left(usage_period,4) = 2023 and t.utility_type = 'W' group by u.usage_period",
    "logic": "This query retrieves the billed water usage for residential customers for each usage period in 2023. It joins several tables to filter for billed water usage, residential customer class, and the specified year. The 'SUM' function calculates the total billing usage for each usage period.",
    "response": "The trend of residential billed water usage for 2023 is displayed below.",
    "chart": {
        "type": "line",
        "x_axis": ["UsagePeriod"],
        "y_axis": ["BillingUsage"],
        "logic": "A line chart is suitable for visualizing the trend of water usage over time.",
    },
},
{
    "question": "Compare 2023 and 2022 residential water revenue",
    "query": "SELECT SUBSTR(bhsrym, 1, 4) AS YEAR, SUM(bhstot) AS BILLINGREVENUE FROM uusehst u JOIN uservice s ON u.service_id = s.service_id JOIN usvctyp t ON s.service_type = t.service_type JOIN ublh b ON u.bill_id = b.bhstr# JOIN ucls c ON b.bhscls = c.uccls WHERE status_type = 'BILLED' AND bhscrc NOT IN ('C', 'S') AND ucdes LIKE '%Residential%' AND t.utility_type = 'W' AND SUBSTR(bhsrym, 1, 4) IN ('2022', '2023') GROUP BY SUBSTR(bhsrym, 1, 4) ORDER BY SUBSTR(bhsrym, 1, 4)",
    "logic": "This query compares the total billed revenue for water services for residential accounts in 2022 and 2023. It joins relevant tables to filter for billed water usage, residential customer class, and the specified years. The 'SUM' function calculates the total revenue from the 'bhstot' column in the 'ublh' table, grouped by the year.",
    "response": "The comparison of residential water revenue between 2023 and 2022 is displayed below.",
    "chart": {
        "type": "bar",
        "x_axis": ["YEAR"],
        "y_axis": ["BILLINGREVENUE"],
        "logic": "A bar chart is suitable for comparing the revenue values between the two years.",
    },
},
{
    "question": "List of active premises with no active service.",
    "query": "select a.UPPRM as PremisesId, UPCDT as PremisesEndDate, SERVICE_ID as ServiceId from UPRM a left outer join USERVICE b on a.UPPRM = b.PREMISES_ID where UPCDT is null and a.UPPRM not in (select distinct bb.PREMISES_ID from uservice bb where PHYSICAL_DISCONNECT_DATE is null)",
    "logic": "This query retrieves a list of premises that are considered active (UPCDT is null) but do not have any active services associated with them. It uses a left outer join between the UPRM and USERVICE tables and filters out premises that have a null PHYSICAL_DISCONNECT_DATE in the USERVICE table, indicating an active service.",
    "response": "Here's the list of active premises with no active service.",
    "chart": {
        "type": "table",
        "x_axis": ["PremisesId", "PremisesEndDate", "ServiceId"],
        "y_axis": [],
        "logic": "A table is appropriate for displaying a list of premises with their corresponding details, including Premises ID, Premises End Date, and Service ID.",
    },
},


#####UNUSED


{
    "question": "Give me a list of premises where the route doesn't match the service.",
    "query": "SELECT a.SERVICE_ID AS ServiceId, a.PREMISES_ID AS PremisesId, a.METER_READING_ROUTE_ID AS ServiceRoute, uprte AS PremisesRoute FROM uservice a JOIN uprm b ON a.PREMISES_ID = b.UPPRM WHERE a.METER_READING_ROUTE_ID <> b.UPRTE",
    "logic": "This query finds premises where the route associated with the service doesn't match the route defined for the premises itself. It joins the 'uservice' and 'uprm' tables on the premises ID and compares the 'METER_READING_ROUTE_ID' (from 'uservice') with the 'UPRTE' (from 'uprm').",
    "response": "Here is a list of premises with mismatched service and premises routes:",
    "chart": {
        "type": "table",
        "x_axis": ["ServiceId", "PremisesId", "ServiceRoute", "PremisesRoute"],
        "y_axis": [],
        "logic": "A table is suitable for presenting the list of premises with mismatched routes.",
    },
},


{
    "question": "How many units of Water Usage were billed with Usage Period June 2023?",
    "query": "SELECT t.utility_type AS UTILITYTYPE, u.usage_period AS USAGEPERIOD, SUM(u.billing_usage) AS BILLINGUSAGE FROM uusehst u JOIN uservice s ON u.service_id = s.service_id JOIN usvctyp t ON s.service_type = t.service_type JOIN ublh b ON u.bill_id = b.bhstr# WHERE status_type = 'BILLED' AND bhscrc NOT IN ('C', 'S') AND usage_period = 202306 AND t.utility_type = 'W' GROUP BY t.utility_type, u.usage_period",
    "logic": "This query retrieves the total units of water usage that were billed for a specific usage period ('202306'). It joins the 'uusehst', 'uservice', 'usvctyp', and 'ublh' tables to filter the relevant data based on the billing status, utility type, and usage period. The 'SUM' function calculates the total billed usage for the specified period, and the 'GROUP BY' clause groups the results by utility type and usage period.",
    "response": "The total units of water usage billed for Usage Period 'June 2023' is displayed below.",
    "chart": {
        "type": "null",
        "x_axis": ["null"],
        "y_axis": ["null"],
        "logic": "Since the question asks for a single total value, no chart is needed.",
    },
},

{
    "question": "List of accounts in cycle '5' that haven't billed for revenue period 202406?",
    "query": "SELECT bhsact AS ACCOUNTID, ubcyc AS BILLCYCLE, MAX(bhsbld) AS LASTBILLDATE FROM ublh a JOIN uactprm b ON a.bhsact = b.account_agreement_id AND a.bhsprm = b.premises JOIN uprm c ON b.premises_id = c.upprm JOIN urte d ON c.uprte = d.ubrte WHERE ubcyc = 5 AND bhscrc NOT IN ('C', 'S') AND end_date IS NULL GROUP BY bhsact, ubcyc HAVING LEFT(MAX(bhsbld), 6) < 202406",
    "logic": "This query retrieves the list of accounts in a specific billing cycle ('5') that haven't billed for the revenue period ('2020406'). It joins the 'ublh', 'uactprm', 'uprm', and 'urte' tables based on various conditions to filter the relevant accounts. The filter conditions ensure that the billing status is not 'C' or 'S', and that the end date is null, indicating active accounts. The 'HAVING' clause filters the results to only include accounts where the maximum billing date is before the specified period ('Y' - June 2024).",
    "response": "The list of accounts in cycle '5' that haven't billed for revenue period '202406' is displayed below.",
    "chart": {
        "type": "null",
        "x_axis": ["null"],
        "y_axis": ["null"],
        "logic": "Since the question asks for a list of accounts, no chart is needed.",
    },
},
{
    "question": "How is residential water revenue trending for 2024?",
    "query": "SELECT b.bhsrym AS REVENUEPERIOD, SUM(b.bhstot) AS BILLINGREVENUE FROM uusehst u JOIN uservice s ON u.service_id = s.service_id JOIN usvctyp t ON s.service_type = t.service_type JOIN ublh b ON u.bill_id = b.bhstr# JOIN ucls c ON b.bhscls = c.uccls WHERE status_type = 'BILLED' AND bhscrc NOT IN ('C', 'S') AND ucdes LIKE '%Residential%' AND LEFT(usage_period, 4) = YEAR(CURRENT DATE) AND t.utility_type = 'W' GROUP BY b.bhsrym",
    "logic": "This query retrieves the residential water revenue trend for the year 2024. It joins multiple tables ('uusehst', 'uservice', 'usvctyp', 'ublh', and 'ucls') to filter the data based on the billing status, utility type, and description. The query sums up the total billed amount (bhstot) for each revenue period within the year 2024. The 'GROUP BY' clause groups the results by revenue period, providing a trend over the year.",
    "response": "The trend of residential water revenue for 2024 is displayed below.",
    "chart": {
        "type": "line",
        "x_axis": ["REVENUEPERIOD"],
        "y_axis": ["BILLINGREVENUE"],
        "logic": "A line chart is appropriate to visualize the trend of water revenue over time, showing how the revenue changes across different periods in 2024.",
    },
}

{'question': 'How is residential water revenue trending for 2024?',
 'query': "SELECT b.bhsrym AS REVENUEPERIOD, SUM(b.bhstot) AS BILLINGREVENUE FROM uusehst u JOIN uservice s ON u.service_id = s.service_id JOIN usvctyp t ON s.service_type = t.service_type JOIN ublh b ON u.bill_id = b.bhstr# JOIN ucls c ON b.bhscls = c.uccls WHERE status_type = 'BILLED' AND bhscrc NOT IN ('C', 'S') AND ucdes LIKE '%Residential%' AND LEFT(usage_period, 4) = YEAR(CURRENT DATE) AND t.utility_type = 'W' GROUP BY b.bhsrym",
 'logic': "This query retrieves the residential water revenue trend for the year 2024. It joins multiple tables ('uusehst', 'uservice', 'usvctyp', 'ublh', and 'ucls') to filter the data based on the billing status, utility type, and description. The query sums up the total billed amount (bhstot) for each revenue period within the year 2024. The 'GROUP BY' clause groups the results by revenue period, providing a trend over the year.",
 'response': 'The trend of residential water revenue f

PKL File

In [68]:
import pickle
# from schema import SAMPLE_QUERIES

my_obj = [table_desc, col_desc, SAMPLE_QUERIES]

In [69]:
pic_file = dest_root+'prequel_ai_4521_schema.pkl'
pickle.dump(my_obj, open(pic_file, 'wb'))

In [70]:
a, b, c = pickle.load(open(pic_file, 'rb'))

In [71]:
c

[{'question': 'List of services with unbilled usage older than 2 months.',
  'query': "select SERVICE_ID as ServiceId, USAGE_PERIOD as UsagePeriod, BILLING_USAGE as BillingUsage, STATUS_TYPE as Status from UUSEHST where BILL_ID is null and STATUS_TYPE = 'READY' and to_date(cast(usage_period as char(6))||'01','YYYYMMDD') < DATE(SUBSTR(CHAR(now()), 1, 7) || '-01') - 2 months",
  'logic': "This query identifies services with unbilled usage that's older than two months. It checks for records in the 'UUSEHST' table where the 'BILL_ID' is null (indicating unbilled usage), the 'STATUS_TYPE' is 'READY', and the usage period is older than two months from the beginning of the current month.",
  'response': "Here's the list of services with unbilled usage older than 2 months.",
  'chart': {'type': 'table',
   'x_axis': ['ServiceId', 'UsagePeriod', 'BillingUsage', 'Status'],
   'y_axis': [],
   'logic': 'A table is appropriate for displaying a list of services with their corresponding details.'}},

Embedding File

In [72]:
import os
import sys

# BASE_PATH = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# print(BASE_PATH)
# sys.path.append(BASE_PATH)

import os
import pickle

import nltk
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv(
    source_root+"description_embedding.csv"
)

# print(df)


nltk.download("stopwords")
model = SentenceTransformer(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True, truncate_dim=256
)

stop_words = set(stopwords.words("english"))

# db_path = os.path.join(
#     os.path.dirname(os.path.dirname(os.path.abspath(__file__))),
#     "sample_database",
# )

df["question_processed"] = df["description"].apply(
    lambda x: " ".join(
        [
            word_token
            for word_token in word_tokenize(x)
            if not word_token.lower() in stop_words
        ]
    )
)

df["embedding"] = df["question_processed"].apply(
    lambda x: model.encode(x, convert_to_tensor=True)
)

with open(dest_root+"prequel_ai_4521_embedding.pkl", "wb") as fp:
    pickle.dump(df, fp)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shubham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/shubham/fabric_main/chat-with-your-data/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>
